In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import zfit
from zfit import z

In [ ]:
import mplhep as hep
hep.style.use(hep.style.CMS)
plt.rcParams.update({
    "text.usetex": True,
})

In [ ]:
from plotting_meta.plotting_meta import bins

from bff_plotting_tools.make_hists import make_sys
from bff_plotting_tools.data_getter import get_data

from bff_signal_model.bff_signal_model import bff_signal_model, reset_params, mu

In [ ]:
era = '2018'
df, lumi = get_data(era)
compute_hesse=True
masses = np.unique(df[df.type=='sig'].mass)
binning_type = 'split'

In [ ]:
obs = zfit.Space("x", limits=bins.calc_bin_range())

In [ ]:
# make histograms
hist_dict = {}
bck_dict = {}
for reg in ['SR1','SR2']:
    print('reg', reg)
    hist_dict[reg] = {} 
    bck_dict[reg] = make_sys(df[(df.type=='bck')], 'DiLepMass', reg)
    for mass in masses:
        mass = int(mass)
        print("\tmass", mass)
        mass_dict = {}
        hist = make_sys(df[(df.mass==mass) & (df.dbs==0.04)], 'DiLepMass', reg)
        normalization = np.sum(hist.nominal)
        sm = bff_signal_model(obs, mu=mass)
        fit_hist = sm.make_hist(bins, normalization, tail_percent=1, constant_percent=.20)  
        mass_dict['mc'] =  hist
        mass_dict['fit'] = fit_hist
        hist_dict[reg][mass] = mass_dict

In [ ]:
# make non-density plots
for reg, reg_dict in hist_dict.items():
    bck = bck_dict[reg]
    for mass, mass_dict in reg_dict.items():
        hist = mass_dict['mc']
        fit_hist = mass_dict['fit']
        fig, ax = plt.subplots(figsize=(10,10))
        bck.draw(ax, color='green', label='MC bck')
        hist.draw(ax, color='red', label='MC')
        fit_hist.draw(ax, color='blue', label='parametric')
        ax.set_yscale('log')
        ax.set_ylabel('Events per GeV')
        ax.set_xlabel('DiLepMass [GeV]')
        ax.set_ylim(bottom=1e-2, top=1e4)
        ax.legend(title = '{} {} GeV'.format(reg, mass))
        fig.savefig('fits/bff/{}_binning_{}_{}_{}.png'.format(binning_type, era, reg, mass))

In [ ]:
# make density plots
for reg, reg_dict in hist_dict.items():
    bck = bck_dict[reg]
    for mass, mass_dict in reg_dict.items():
        hist = mass_dict['mc']
        fit_hist = mass_dict['fit']
        fig, ax = plt.subplots(figsize=(10,10))
        bck.make_density_hist().draw(ax, color='green', label='MC bck')
        hist.make_density_hist().draw(ax, color='red', label='MC')
        fit_hist.make_density_hist().draw(ax, color='blue', label='parametric')
        ax.set_yscale('log')
        ax.set_ylabel('Events per GeV')
        ax.set_xlabel('DiLepMass [GeV]')
        ax.set_ylim(bottom=1e-2, top=1e4)
        ax.legend(title = '{} {} GeV'.format(reg, mass))
        fig.savefig('fits/bff/{}_binning_density_{}_{}_{}.png'.format(binning_type, era, reg, mass))

In [ ]:
# make density plots
for reg, reg_dict in hist_dict.items():
    for mass, mass_dict in reg_dict.items():
        hist = mass_dict['mc']
        fit_hist = mass_dict['fit']
        fig, ax = plt.subplots(figsize=(10,10))
        hist.calc_ratio(hist.nominal).draw(ax, color='red', label='MC')
        fit_hist.calc_ratio(fit_hist.nominal).draw(ax, color='blue', label='parametric')
        #ax.set_yscale('log')
        ax.set_ylabel('Ratio with nominal')
        ax.set_xlabel('DiLepMass [GeV]')
        ax.set_ylim(bottom=0, top=2)
        ax.legend(title = '{} {} GeV'.format(reg, mass))
        fig.savefig('fits/bff/{}_binning_ratio_{}_{}_{}.png'.format(binning_type, era, reg, mass))